In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
xtrain = pd.read_csv("training_set_features.csv")
ytrain = pd.read_csv("training_set_labels.csv")
testSet = pd.read_csv("test_set_features.csv")

In [4]:
print(xtrain.shape, ytrain.shape, testSet.shape)

(26707, 36) (26707, 3) (26708, 36)


# Filter useful data

In [6]:
# cols to remove: health_insurance, employment industry and occupation

# cols to separate: doctor_recc xyz and seasonal, opinion effective/risk/sick xyz and seasonal

def separate(df):
    colsReqdxyz = ['respondent_id','doctor_recc_xyz','health_insurance',
        'chronic_med_condition', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
        'opinion_xyz_sick_from_vacc']
    colsReqdseas = ['respondent_id','doctor_recc_seasonal','health_insurance',
        'chronic_med_condition', 'opinion_seas_vacc_effective','opinion_seas_risk', 'opinion_seas_sick_from_vacc']
    
    return df[colsReqdseas], df[colsReqdxyz]

xtrainseas, xtrainxyz = separate(xtrain)

ytrainseas, ytrainxyz = ytrain["seasonal_vaccine"], ytrain["xyz_vaccine"]

# Handle missing values

In [7]:
from sklearn.impute import SimpleImputer
def fillMissing(df):
    imp = SimpleImputer(strategy = "most_frequent")
    cols = df.columns
    fill = imp.fit_transform(df)
    return pd.DataFrame(fill, columns = cols, index = df.index)

xtrainxyz = fillMissing(xtrainxyz)
xtrainseas = fillMissing(xtrainseas)

# Encoding: DO NOT USE, it was for old multinomialNB trial

In [8]:
# do one hot encoding for all
# from sklearn.preprocessing import OneHotEncoder

# def encode(df):
#     toEncode = ['age_group','education','race','sex','income_poverty','marital_status','rent_or_own','employment_status','hhs_geo_region','census_msa']

#     # one hot encoder
#     encoder = OneHotEncoder()
#     for i in toEncode:
#         result = encoder.fit_transform(df[[i]])
#         df = df.join(pd.DataFrame(result.toarray(), columns = encoder.categories_))
    
#     # drop the pre-encoded cols
#     df.drop(columns = toEncode, inplace = True)

#     # convert the tuple type column names to string
#     df.columns = [x[0] if type(x) == tuple else x for x in df.columns]

#     # if dtype is still object, make it int
#     for col in df.select_dtypes(include = ['object']).columns:
#         df[col] = df[col].astype(int)
    
#     return df

# xtrainxyz = encode(xtrainxyz)
# xtrainseas = encode(xtrainseas)

# Fitting models

In [9]:
from sklearn.model_selection import train_test_split

x = xtrainxyz
y = ytrainxyz
trainX1, testX1, trainY1, testY1 = train_test_split(x, y, test_size = 0.2)
x = xtrainseas
y = ytrainseas
trainX2, testX2, trainY2, testY2 = train_test_split(x, y, test_size = 0.2)

In [10]:
from sklearn.linear_model import LogisticRegression

modelxyz = LogisticRegression(max_iter = 500, random_state = 1)
modelxyz.fit(trainX1, trainY1)
modelxyz.score(testX1, testY1)

C:\Users\susha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8290902283788844

In [11]:
modelseas = LogisticRegression(max_iter = 500, random_state = 2)
modelseas.fit(trainX2, trainY2)
modelseas.score(testX2, testY2)

C:\Users\susha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7502807937102209

# Getting results

In [12]:
# filter useful cols
testSeasonal, testXYZ = separate(testSet)

# handle missing values
testSeasonal = fillMissing(testSeasonal)
testXYZ = fillMissing(testXYZ)

# encode text cols
# testSeasonal = encode(testSeasonal)
# testXYZ = encode(testXYZ)

In [13]:
h1n1 = modelxyz.predict(testXYZ)
seasonal = modelseas.predict(testSeasonal)

result = pd.DataFrame( {
    "h1n1_vaccine" : h1n1,
    "seasonal_vaccine" : seasonal
} ,index = testSeasonal["respondent_id"])

result

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707.0,0,0
26708.0,0,0
26709.0,0,1
26710.0,0,1
26711.0,0,1
...,...,...
53410.0,0,1
53411.0,0,0
53412.0,0,0


In [14]:
result.to_csv("result.csv")